<a href="https://colab.research.google.com/github/kojurkin/NonparametricsStat/blob/main/NonparametricsStat_test_inside.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.special import gamma, gammaln
from scipy.integrate import quad

In [ ]:
def empirical_cdf(x, sample):
    n = len(sample)
    return np.sum(sample <= x) / n

In [ ]:
def bessel_mod(v, z, k_max=100, tolerance=1e-6):
    total_sum = 0.0
    z_half = z / 2.0
    for k in range(k_max):
        log_term = (v + 2 * k) * np.log(abs(z / 2.0)) - gammaln(k + 1) - gammaln(k + v + 1)
        term = np.exp(log_term)
        total_sum += term
        if abs(term) < tolerance:
            break
    return total_sum

In [ ]:
def integrand(y, t, j):
  return np.exp((t/(8*(y**2 + 1))) - (((4*j+1)**2 * np.pi**2 * y**2)/(8*t)))

In [ ]:
def compute_ranks(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    # (элемент, выборка, исходный_индекс)
    combined = [(x, 0, i) for i, x in enumerate(sample1)] + [(x, 1, i) for i, x in enumerate(sample2)]
    sorted_data = sorted(combined, key=lambda x: x[0])
    n = len(combined)
    ranks = [0] * n
    i = 0
    while i < n:
        start = i
        current_value = sorted_data[i][0]
        count = 0
        while i < n and sorted_data[i][0] == current_value:
            i += 1
            count += 1
        avg_rank = (start + 1 + start + count) / 2
        for j in range(start, i):
            ranks[sorted_data[j][2] + (sorted_data[j][1] * len(sample1))] = avg_rank
    ranks1 = []
    ranks2 = []
    for item in sorted(combined, key=lambda x: x[2]):
        rank_index = item[2] + (item[1] * len(sample1))
        if item[1] == 0:
            ranks1.append(ranks[rank_index])
        else:
            ranks2.append(ranks[rank_index])
    return np.array(ranks1), np.array(ranks2)

In [ ]:
def a1function(t, j_max=1000, tolerance=1e-16, round_to_5 = True):
    if t <= 0:
        return 0.0
    total_sum = 0.0
    sqrt_2t = np.sqrt(2 * t)

    for j in range(j_max):
        term1 = gamma(j + 0.5) * np.sqrt(4 * j + 1)
        term2 = gamma(0.5) * gamma(j + 1)
        exponent = np.exp(-(4 * j + 1)**2 / (16 * t))
        arg = (4 * j + 1)**2 / (16 * t)
        bessel_diff = bessel_mod(-0.25, arg, k_max=173, tolerance=tolerance) - bessel_mod(0.25, arg, k_max=173, tolerance=tolerance)

        term = (term1 / term2) * exponent * bessel_diff

        total_sum += term

        if abs(term) < tolerance:
            break
    result = total_sum / np.sqrt(2 * t)

    if round_to_5:
        result = round(result, 5)
    return result

In [ ]:
def a2function(t, j_max=1000, tolerance=1e-16, round_to_5 = True):
    if t <= 0:
        return 0.0
    total_sum = 0.0

    for j in range(j_max):
        term1 = gamma(j + 0.5) * (4 * j + 1)
        term2 = gamma(0.5) * gamma(j + 1)
        exponent = np.exp(-1 * ((4 * j + 1)**2 * np.pi**2) / (8 * t))
        integral_exponent, tolerance_message = quad(integrand, 0, np.inf, args=(t, j))

        term = (-1)**j * (term1 / term2) * exponent * integral_exponent

        total_sum += term

        if abs(term) < tolerance:
            break

    result = total_sum * (np.sqrt(2 * np.pi) / t)
    if round_to_5:
        result = round(result, 5)
#    print(tolerance_message)
    return result

In [ ]:
def w_squared_test(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    n = len(sample1)
    sample2_sorted = np.sort(sample2)
    sample2_cdf = np.vectorize(lambda x: empirical_cdf(x, sample2))
    U = sample2_cdf(sample1)
    U_sorted = np.sort(U)
    sum_term = 0
    for i in range(n):
        diff = U_sorted[i] - (2 * (i + 1) - 1) / (2 * n)
        sum_term += diff ** 2
    w_squared = ((1 / (12 * n)) + sum_term)
    p_value = 1 - a1function(w_squared)
    return w_squared, p_value

In [ ]:
def lehmann_rosenblatt_test(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    m, n = len(sample1), len(sample2)
    if m > n:
        tmp = sample2
        sample2 = sample1
        sample1 = tmp
        m, n = len(sample1), len(sample2)
    ranks1, ranks2 = compute_ranks(sample1, sample2)
    sum1 = 0
    sum2 = 0
    ranks1 = sorted(ranks1)
    ranks2 = sorted(ranks2)
    for i in range(len(ranks1)):
      sum1 += (ranks1[i] - (i + 1))**2
    for i in range(len(ranks2)):
      sum2 += (ranks2[i] - (i + 1))**2
    T = (n * sum1 + m * sum2) / (m * n * (m + n)) - ((4 * m * n - 1) / (6 * (m + n)))
    p_value = 1 - a1function(T)
    return T, p_value

In [ ]:
def anderson_darling_test(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    m, n = len(sample1), len(sample2)
    if m > n:
      tmp = sample2
      sample2 = sample1
      sample1 = tmp

    combined = np.concatenate([sample1, sample2])
    sorted_combined = np.sort(combined)

    ranks1, ranks2 = compute_ranks(sample1, sample2)

    M = []
    for j in range(m + n):
        value = sorted_combined[j]
        M_j = sum(1 for x in sample1 if x <= value)
        M.append(M_j)
    total = 0.0
    for i in range(1, m + n):
        term = ((M[i-1] * (m + n) - m*i) ** 2) / (i * (m + n - i))
        total += term

    A = total / (m * n)
    p_value = 1 - a2function(A)
    return A, p_value

In [ ]:
def kolmogorov_smirnov_test(data1, data2):
    data1 = np.sort(data1)
    data2 = np.sort(data2)
    n1 = len(data1)
    n2 = len(data2)
    data_all = np.concatenate([data1, data2])
    cdf1 = np.searchsorted(data1, data_all, side='right') / np.float64(n1)
    cdf2 = np.searchsorted(data2, data_all, side='right') / np.float64(n2)
    d = np.float64(np.max(np.abs(cdf1 - cdf2)))
    return d, np.clip(kolmogorov_distribution(d, n1, n2), 0.0, 1.0),

In [ ]:
def kolmogorov_distribution(d, n1, n2):
    d = float(d)
    n1 = int(n1)
    n2 = int(n2)

    if n1 <= 10 and n2 <= 10:
        m, n = n1, n2
        k = int(np.ceil(d * n * m))
        if k == 0:
            return 1.0
        dp = np.zeros((n + 1, m + 1, k + 1), dtype=np.float64)
        dp[0, 0, 0] = 1.0
        for i in range(n + 1):
            for j in range(m + 1):
                for t in range(k + 1):
                    if dp[i, j, t] == 0:
                        continue
                    diff = abs(i / np.float64(n) - j / np.float64(m))
                    if diff * n * m > t:
                        continue
                    if i < n:
                        dp[i + 1, j, t] += dp[i, j, t] / np.float64(n)
                    if j < m:
                        dp[i, j + 1, t] += dp[i, j, t] / np.float64(m)
        p = 0.0
        for t in range(k):
            p += dp[n, m, t]
        return np.clip(1 - p, 0.0, 1.0)
    else:
        m, n = sorted([float(n1), float(n2)], reverse=True)
        en = m * n / (m + n)
        z = d * np.sqrt(en)
        z = np.asarray(z, dtype=np.float64)
        p = np.zeros_like(z)
        mask_non_positive = z <= 0
        p[mask_non_positive] = 1.0
        mask_inf = np.isinf(z)
        p[mask_inf] = 0.0
        mask_valid = (~mask_non_positive) & (~mask_inf)
        z_valid = z[mask_valid]
        if z_valid.size > 0:
            mask_small = z_valid < 1.18
            if np.any(mask_small):
                sqrt_2pi = np.sqrt(2 * np.pi)
                z_small = z_valid[mask_small]
                p_small_sum = np.zeros_like(z_small)
                for k in range(1, 1000):
                    term = -(2*k - 1)**2 * np.pi**2 / (8 * z_small**2)
                    if np.any(term < -708.3964185322641):
                        break
                    current = np.where(term == -np.inf, 0.0, np.exp(np.clip(term, -708.3964185322641, 709.782712893384)))
                    p_small_sum += current
                    if np.all(np.abs(current) < 1e-30):
                        break
                raw_p_small = 1.0 - sqrt_2pi / z_small * p_small_sum
                p_small = np.clip(raw_p_small, 0.0, 1.0)
                if z.ndim == 0:
                    if mask_small:
                        p = p_small
                else:
                    p[mask_valid & mask_small] = p_small
            mask_large = ~mask_small
            if np.any(mask_large):
                z_large = z_valid[mask_large]
                p_large_sum = np.zeros_like(z_large)
                for k in range(1, 1000):
                    term = -2 * k**2 * z_large**2
                    if np.all(term < -708.3964185322641):
                        break
                    sign = 1.0 if k % 2 == 1 else -1.0
                    current = sign * np.where(term == -np.inf, 0.0, np.exp(np.clip(term, -708.3964185322641, 709.782712893384)))
                    p_large_sum += current
                    if np.all(np.abs(current) < 1e-30):
                        break
                raw_p_large = 2.0 * p_large_sum
                p_large = np.clip(raw_p_large, 0.0, 1.0)
                if z.ndim == 0:
                    if mask_large:
                        p = p_large
                else:
                    p[mask_valid & mask_large] = p_large
        return p.item() if z.ndim == 0 else p

Тестирование

In [ ]:
import pandas as pd
ds0 = pd.read_excel('/content/AD.ds3.xlsx')
ds378 = ds0[['v78', 'G']].copy()
ds378.columns = ['s78', 'Factor']
ds378['Factor'] = ds378['Factor'].astype('category')
print(ds378.head(5))
sample781 = ds378[ds378['Factor'] == 1]
sample782 = ds378[ds378['Factor'] == 2]
sample783 = ds378[ds378['Factor'] == 3]
sample784 = ds378[ds378['Factor'] == 4]

print(sample781)
print(sample782)
print(sample783)
print(sample784)


         s78 Factor
0  17.126126      1
1  18.175637      3
2  11.518727      3
3  11.382667      1
4   8.841300      1
            s78 Factor
0     17.126126      1
3     11.382667      1
4      8.841300      1
5     28.194540      1
7     10.210544      1
...         ...    ...
1391  28.597946      1
1392  16.530179      1
1396  13.914268      1
1397  14.255363      1
1398  14.377280      1

[350 rows x 2 columns]
            s78 Factor
8     35.571224      2
29     8.464201      2
30     8.188931      2
31     2.551928      2
32    27.362216      2
...         ...    ...
1380  30.152727      2
1384  36.423330      2
1388   7.655273      2
1393  11.133238      2
1394  10.827193      2

[350 rows x 2 columns]
            s78 Factor
1     18.175637      3
2     11.518727      3
6     22.342715      3
11    27.853343      3
12    19.330229      3
...         ...    ...
1383  42.838541      3
1389   5.402262      3
1390  27.386909      3
1395  18.627133      3
1399   7.182120      3

[35

In [ ]:
sample1 = sample781['s78']
sample2 = sample782['s78']
sample3 = sample783['s78']
sample4 = sample784['s78']
my_sample_list = [sample1, sample2, sample3, sample4]
from itertools import combinations
results = []

In [ ]:
from scipy.stats import ks_2samp, anderson_ksamp
from scipy import stats
import time

In [ ]:
print("---Андерсона-Дарлинга 1-2---")
start_time = time.time()
ad_scipy_stat, ad_scipy_crit, ad_scipy_p = anderson_ksamp([sample1, sample2], method=stats.PermutationMethod())
ad_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ad_scipy_time," Статистика ", ad_scipy_stat, " p-value ", ad_scipy_p)

start_time = time.time()
ad_stat, ad_p = anderson_darling_test(sample1, sample2)
ad_time = time.time() - start_time
print("Мои результаты: Время ", ad_time," Статистика ", ad_stat, " p-value ", ad_p)

print("---Андерсона-Дарлинга 1-3---")
start_time = time.time()
ad_scipy_stat, ad_scipy_crit, ad_scipy_p = anderson_ksamp([sample1, sample3], method=stats.PermutationMethod())
ad_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ad_scipy_time," Статистика ", ad_scipy_stat, " p-value ", ad_scipy_p)

start_time = time.time()
ad_stat, ad_p = anderson_darling_test(sample1, sample3)
ad_time = time.time() - start_time
print("Мои результаты: Время ", ad_time," Статистика ", ad_stat, " p-value ", ad_p)

print("---Андерсона-Дарлинга 1-4---")
start_time = time.time()
ad_scipy_stat, ad_scipy_crit, ad_scipy_p = anderson_ksamp([sample1, sample4], method=stats.PermutationMethod())
ad_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ad_scipy_time," Статистика ", ad_scipy_stat, " p-value ", ad_scipy_p)

start_time = time.time()
ad_stat, ad_p = anderson_darling_test(sample1, sample4)
ad_time = time.time() - start_time
print("Мои результаты: Время ", ad_time," Статистика ", ad_stat, " p-value ", ad_p)

print("---Андерсона-Дарлинга 2-3---")
start_time = time.time()
ad_scipy_stat, ad_scipy_crit, ad_scipy_p = anderson_ksamp([sample2, sample3], method=stats.PermutationMethod())
ad_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ad_scipy_time," Статистика ", ad_scipy_stat, " p-value ", ad_scipy_p)

start_time = time.time()
ad_stat, ad_p = anderson_darling_test(sample2, sample3)
ad_time = time.time() - start_time
print("Мои результаты: Время ", ad_time," Статистика ", ad_stat, " p-value ", ad_p)

print("---Андерсона-Дарлинга 2-4---")
start_time = time.time()
ad_scipy_stat, ad_scipy_crit, ad_scipy_p = anderson_ksamp([sample2, sample4], method=stats.PermutationMethod())
ad_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ad_scipy_time," Статистика ", ad_scipy_stat, " p-value ", ad_scipy_p)

start_time = time.time()
ad_stat, ad_p = anderson_darling_test(sample2, sample4)
ad_time = time.time() - start_time
print("Мои результаты: Время ", ad_time," Статистика ", ad_stat, " p-value ", ad_p)

print("---Андерсона-Дарлинга 3-4---")
start_time = time.time()
ad_scipy_stat, ad_scipy_crit, ad_scipy_p = anderson_ksamp([sample3, sample4], method=stats.PermutationMethod())
ad_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ad_scipy_time," Статистика ", ad_scipy_stat, " p-value ", ad_scipy_p)

start_time = time.time()
ad_stat, ad_p = anderson_darling_test(sample3, sample4)
ad_time = time.time() - start_time
print("Мои результаты: Время ", ad_time," Статистика ", ad_stat, " p-value ", ad_p)

---Андерсона-Дарлинга 1-2---
Результаты SciPy: Время  3.151284694671631  Статистика  -0.5939009471634233  p-value  0.6989
Мои результаты: Время  0.03559279441833496  Статистика  0.5491714508044634  p-value  0.6972700000000001
---Андерсона-Дарлинга 1-3---
Результаты SciPy: Время  5.6431450843811035  Статистика  0.2049604153708288  p-value  0.277
Мои результаты: Время  0.10778093338012695  Статистика  1.1591795010732562  p-value  0.28358000000000005
---Андерсона-Дарлинга 1-4---
Результаты SciPy: Время  2.258624792098999  Статистика  11.82766735984587  p-value  0.0001
Мои результаты: Время  0.03600955009460449  Статистика  9.934656085753927  p-value  9.99999999995449e-06
---Андерсона-Дарлинга 2-3---
Результаты SciPy: Время  2.241257905960083  Статистика  -0.34137344452599894  p-value  0.5265
Мои результаты: Время  0.03602790832519531  Статистика  0.740705260814037  p-value  0.52578
---Андерсона-Дарлинга 2-4---
Результаты SciPy: Время  2.2195279598236084  Статистика  14.339568617004662  p-

In [ ]:
print("---Лемана-Розенблата 1-2---")
start_time = time.time()
lr_stat, lr_p = lehmann_rosenblatt_test(sample1, sample2)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Лемана-Розенблата 1-3---")
start_time = time.time()
lr_stat, lr_p = lehmann_rosenblatt_test(sample1, sample3)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Лемана-Розенблата 1-4---")
start_time = time.time()
lr_stat, lr_p = lehmann_rosenblatt_test(sample1, sample4)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Лемана-Розенблата 2-3---")
start_time = time.time()
lr_stat, lr_p = lehmann_rosenblatt_test(sample2, sample3)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Лемана-Розенблата 2-4---")
start_time = time.time()
lr_stat, lr_p = lehmann_rosenblatt_test(sample2, sample4)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Лемана-Розенблата 3-4---")
start_time = time.time()
lr_stat, lr_p = lehmann_rosenblatt_test(sample3, sample4)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

---Лемана-Розенблата 1-2---
Мои результаты: Время  0.004612922668457031  Статистика  0.0821714285714279  p-value  0.6793899999999999
---Лемана-Розенблата 1-3---
Мои результаты: Время  0.029995441436767578  Статистика  0.2320326530612249  p-value  0.21331
---Лемана-Розенблата 1-4---
Мои результаты: Время  0.009383916854858398  Статистика  0.9630265306122539  p-value  0.0030000000000000027
---Лемана-Розенблата 2-3---
Мои результаты: Время  0.022568464279174805  Статистика  0.11284489795919228  p-value  0.52605
---Лемана-Розенблата 2-4---
Мои результаты: Время  0.020554780960083008  Статистика  1.1095448979591822  p-value  0.0013699999999999823
---Лемана-Розенблата 3-4---
Мои результаты: Время  0.005512237548828125  Статистика  0.8962510204081724  p-value  0.0043199999999999905


In [ ]:
print("---Омега-квадрат 1-2---")
start_time = time.time()
lr_stat, lr_p = w_squared_test(sample1, sample2)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Омега-квадрат 1-3---")
start_time = time.time()
lr_stat, lr_p = w_squared_test(sample1, sample3)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Омега-квадрат 1-4---")
start_time = time.time()
lr_stat, lr_p = w_squared_test(sample1, sample4)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Омега-квадрат 2-3---")
start_time = time.time()
lr_stat, lr_p = w_squared_test(sample2, sample3)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Омега-квадрат 2-4---")
start_time = time.time()
lr_stat, lr_p = w_squared_test(sample2, sample4)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

print("---Омега-квадрат 3-4---")
start_time = time.time()
lr_stat, lr_p = w_squared_test(sample3, sample4)
lr_time = time.time() - start_time
print("Мои результаты: Время ", lr_time," Статистика ", lr_stat, " p-value ", lr_p)

---Омега-квадрат 1-2---
Мои результаты: Время  0.06137895584106445  Статистика  0.16377687074829952  p-value  0.35039
---Омега-квадрат 1-3---
Мои результаты: Время  0.04601407051086426  Статистика  0.4633523809523799  p-value  0.049409999999999954
---Омега-квадрат 1-4---
Мои результаты: Время  0.04735898971557617  Статистика  1.9255727891156433  p-value  2.0000000000020002e-05
---Омега-квадрат 2-3---
Мои результаты: Время  0.047342538833618164  Статистика  0.225131972789116  p-value  0.22363999999999995
---Омега-квадрат 2-4---
Мои результаты: Время  0.051891326904296875  Статистика  2.218601360544214  p-value  0.0
---Омега-квадрат 3-4---
Мои результаты: Время  0.041686296463012695  Статистика  1.792021768707485  p-value  4.0000000000040004e-05


In [ ]:
print("---Колмогорова-Смирнова---")
start_time = time.time()
ks_scipy_stat, ks_scipy_p = ks_2samp(sample1, sample2)
ks_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ks_scipy_time, " Статистика ", ks_scipy_stat, " p-value ", ks_scipy_p)

start_time = time.time()
ks_stat, ks_p = kolmogorov_smirnov_test(sample1, sample2)
ks_time = time.time() - start_time
print("Мои результаты: Время ", ks_time, " Статистика ", ks_stat, " p-value ", ks_p)

print("---Колмогорова-Смирнова---")
start_time = time.time()
ks_scipy_stat, ks_scipy_p = ks_2samp(sample1, sample3)
ks_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ks_scipy_time, " Статистика ", ks_scipy_stat, " p-value ", ks_scipy_p)

start_time = time.time()
ks_stat, ks_p = kolmogorov_smirnov_test(sample1, sample3)
ks_time = time.time() - start_time
print("Мои результаты: Время ", ks_time, " Статистика ", ks_stat, " p-value ", ks_p)

print("---Колмогорова-Смирнова---")
start_time = time.time()
ks_scipy_stat, ks_scipy_p = ks_2samp(sample1, sample4)
ks_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ks_scipy_time, " Статистика ", ks_scipy_stat, " p-value ", ks_scipy_p)

start_time = time.time()
ks_stat, ks_p = kolmogorov_smirnov_test(sample1, sample4)
ks_time = time.time() - start_time
print("Мои результаты: Время ", ks_time, " Статистика ", ks_stat, " p-value ", ks_p)

print("---Колмогорова-Смирнова---")
start_time = time.time()
ks_scipy_stat, ks_scipy_p = ks_2samp(sample2, sample3)
ks_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ks_scipy_time, " Статистика ", ks_scipy_stat, " p-value ", ks_scipy_p)

start_time = time.time()
ks_stat, ks_p = kolmogorov_smirnov_test(sample2, sample3)
ks_time = time.time() - start_time
print("Мои результаты: Время ", ks_time, " Статистика ", ks_stat, " p-value ", ks_p)

print("---Колмогорова-Смирнова---")
start_time = time.time()
ks_scipy_stat, ks_scipy_p = ks_2samp(sample2, sample4)
ks_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ks_scipy_time, " Статистика ", ks_scipy_stat, " p-value ", ks_scipy_p)

start_time = time.time()
ks_stat, ks_p = kolmogorov_smirnov_test(sample2, sample4)
ks_time = time.time() - start_time
print("Мои результаты: Время ", ks_time, " Статистика ", ks_stat, " p-value ", ks_p)

print("---Колмогорова-Смирнова---")
start_time = time.time()
ks_scipy_stat, ks_scipy_p = ks_2samp(sample3, sample4)
ks_scipy_time = time.time() - start_time
print("Результаты SciPy: Время ", ks_scipy_time, " Статистика ", ks_scipy_stat, " p-value ", ks_scipy_p)

start_time = time.time()
ks_stat, ks_p = kolmogorov_smirnov_test(sample3, sample4)
ks_time = time.time() - start_time
print("Мои результаты: Время ", ks_time, " Статистика ", ks_stat, " p-value ", ks_p)


---Колмогорова-Смирнова---
Результаты SciPy: Время  0.0015175342559814453  Статистика  0.05714285714285714  p-value  0.6179086766712222
Мои результаты: Время  0.0006899833679199219  Статистика  0.05714285714285716  p-value  0.6171949793981644
---Колмогорова-Смирнова---
Результаты SciPy: Время  0.0012993812561035156  Статистика  0.08285714285714285  p-value  0.1809122045806726
Мои результаты: Время  0.0008060932159423828  Статистика  0.08285714285714285  p-value  0.1807843407352241
---Колмогорова-Смирнова---
Результаты SciPy: Время  0.0011665821075439453  Статистика  0.14857142857142858  p-value  0.0008673277143634175
Мои результаты: Время  0.0005559921264648438  Статистика  0.14857142857142858  p-value  0.0008826629809542174
---Колмогорова-Смирнова---
Результаты SciPy: Время  0.0011322498321533203  Статистика  0.06  p-value  0.5550396520563093
Мои результаты: Время  0.00060272216796875  Статистика  0.06  p-value  0.5543843284001296
---Колмогорова-Смирнова---
Результаты SciPy: Время  0.